In [1]:
import spacy
nlp = spacy.load('de_core_news_sm')
from spacy.tokenizer import Tokenizer
from spacy.tokens import Doc
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

C:\Users\marku\anaconda3\envs\final\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\marku\anaconda3\envs\final\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\marku\anaconda3\envs\final\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\marku\anaconda3\envs\final\lib\site-packages\tensorflow\python\f

In [2]:
words = []
annotations = []

try:
    fh = open("/Users/marku/Downloads/corpus_part_Deli.txt", encoding="UTF-8")
    
    for line_num, line_txt in enumerate(fh, start=1):
        if line_num %2 == 1:
            words.append(line_txt.strip())
        else:
            annotations.append(line_txt.strip())
    fh.close()
except IOError:
    print("Could not read file")

print("# Äußerungen: ", len(words))
print("# Annotationen: ", len(annotations))

# Äußerungen:  349
# Annotationen:  349


In [3]:
text = []
incorrect_annotations = []
doc_id = 0
annotations_in_use = ""
initial_tags = []

for doc in nlp.pipe(words):
    tokens = []
    for token in doc:
        tokens.append(token.lemma_)
    tags = annotations[doc_id].split()
    initial_tags.append(tags[0])
    if len(tokens) == len(tags):
        for t in range(len(tokens)):
            text.append(tokens[t].lower() + " " + tags[t])
            annotations_in_use = annotations_in_use + " " + annotations[doc_id]
    else:
        print(2*(doc_id+1), len(tokens), len(tags))
        incorrect_annotations.append(doc_id)
    doc_id = doc_id + 1
print("Fehler in der Annotation bei folgenden Äußerungen: ", incorrect_annotations)
print(sorted(set(annotations_in_use.split())))

Fehler in der Annotation bei folgenden Äußerungen:  []
['B-Beurteilung,Bewertung,Gutachten,Wertung', 'B-Dauer', 'B-Ende,Endstück', 'B-Essen,Esswaren,Eßwaren,Lebensmittel,Nahrung,Nahrungsmittel,Speisen', 'B-Forum,Plattform', 'B-GemüseSahnesoße', 'B-Gericht:Gemüsesoße', 'B-Gericht:Sauce', 'B-Geschmacksträger', 'B-Gewürz', 'B-Kategorie:Aufläufe', 'B-Kategorie:Curry', 'B-Kategorie:Flüssigkeit', 'B-Kategorie:Gemüse', 'B-Kategorie:Gewürze', 'B-Kategorie:Kuchen', 'B-Kategorie:Milchprodukte', 'B-Kategorie:Pasta', 'B-Kategorie:Resteküche', 'B-Kategorie:Rezept', 'B-Kategorie:Rezepte', 'B-Kategorie:Rührei', 'B-Kategorie:Sahne', 'B-Kategorie:Sahnesauce', 'B-Kategorie:Salat', 'B-Kategorie:Sauce', 'B-Kategorie:Video', 'B-Kategorie:Zutaten', 'B-Knollen-Ziest', 'B-Kommentar,Kommentierung', 'B-Kraft', 'B-Liter,Maß', 'B-Menge', 'B-Menge:Brühe', 'B-Menge:Gramm', 'B-Menge:Milliliter', 'B-Menge:Personen', 'B-Menge:Zentimeter', 'B-Negation', 'B-Personen', 'B-Pfeffer', 'B-Punkt', 'B-Rezept', 'B-Rezept,Zubere

In [4]:
wordsLemma = []
for doc in nlp.pipe(words):
    tokens = []
    for token in doc:
        tokens.append(token.lemma_.lower())
    wordsLemma.append(tokens)

In [5]:
dataset = []
countSentence = 0
for sentence in wordsLemma:
    countElement = 0
    anno = re.sub(':[A-Z|a-z|0-9|ä|ü|ö|ß]*', '', annotations[countSentence]).split()
    for element in sentence:
        tuples = []
        tuples.append(countSentence)
        tuples.append(element)
        tuples.append(anno[countElement])
        countElement += 1
        dataset.append(tuples)
    countSentence += 1

In [6]:
sentenceLengthList = []
count = 0
lastNumber = 0
for element in dataset:
    if(element[0] == lastNumber):
        count += 1
    else:
        sentenceLengthList.append(count)
        count = 1
        lastNumber += 1
sentenceLengthList.append(count)

In [7]:
maxSentenceLength = 50 #Post Padding auf Satzlänge 50
newDataset = []
countSentence = 0
countIndex = 0
count = 0
for liste in sentenceLengthList:
    for i in range(maxSentenceLength):
        tuples = []
        if  i < liste:
            tuples.append(countSentence)
            tuples.append(dataset[count][1])
            tuples.append(dataset[count][2])
            newDataset.append(tuples)
            count += 1
        else:
            tuples.append(countSentence)
            tuples.append('PADword')
            tuples.append('O')
            newDataset.append(tuples)
    countSentence += 1  

In [8]:
df = DataFrame (newDataset, columns=['Sentence', 'Word', 'Annotation'])

In [9]:
count = 0 #Anlegen einer Liste mit den X-Werten
listX = []
tmp = []
for i in range (len(df['Word'])): 
    if count < maxSentenceLength:
        tmp.append(newDataset[i][1])
    else:
        count = 0
        listX.append(tmp)
        tmp = []
        tmp.append(newDataset[i][1])
    count += 1

In [10]:
count = 0 #Anlegen einer Liste mit den Y-Werten
listY = []
tmp = []
for i in range (len(df['Annotation'])): 
    if count < maxSentenceLength:
        tmp.append(newDataset[i][2])
    else:
        count = 0
        listY.append(tmp)
        tmp = []
        tmp.append(newDataset[i][2])
    count += 1

In [11]:
dataset2 = [] #Neues Dataset da nun nur noch Values von Tag
countSentence = 0
for sentence in wordsLemma:
    countElement = 0
    anno = re.sub('[A-Z]-[A-Z|a-z|-|ä|Ä|ü|Ü|ö|ö|ß|,|0-9]*:', '', annotations[countSentence]).split()
    for element in sentence:
        tuples = []
        tuples.append(countSentence)
        tuples.append(element)
        tuples.append(anno[countElement])
        countElement += 1
        dataset2.append(tuples)
    countSentence += 1

In [12]:
tmp1 = [] #Tags ohne Value durch 'O' ersetzen
for liste in dataset2:
    tmp2 = []
    for element in liste:
        if isinstance(element, str):
            if element.startswith('B-'):
                tmp2.append('O')
            elif element.startswith('I-'):
                tmp2.append('O')
            else:
                tmp2.append(element)
        else:
            tmp2.append(element)
    tmp1.append(tmp2)
dataset2 = tmp1

In [13]:
maxSentenceLength = 50 #Post Padding
newDataset2 = []
countSentence = 0
countIndex = 0
count = 0
for liste in sentenceLengthList:
    for i in range(maxSentenceLength):
        tuples = []
        if  i < liste:
            tuples.append(countSentence)
            tuples.append(dataset2[count][1])
            tuples.append(dataset2[count][2])
            newDataset2.append(tuples)
            count += 1
        else:
            tuples.append(countSentence)
            tuples.append('PADword')
            tuples.append('O')
            newDataset2.append(tuples) 
    countSentence += 1  

In [14]:
df2 = DataFrame (newDataset2, columns=['Sentence', 'Word', 'Annotation'])

In [15]:
count = 0 #Anlegen einer Liste mit den X-Werten
listX2 = []
tmp = []
for i in range (len(df2['Word'])): 
    if count < maxSentenceLength:
        tmp.append(newDataset2[i][1])
    else:
        count = 0
        listX2.append(tmp)
        tmp = []
        tmp.append(newDataset2[i][1])
    count += 1

In [16]:
count = 0 #Anlegen einer Liste mit den Y-Werten
listY2 = []
tmp = []
for i in range (len(df2['Annotation'])): 
    if count < maxSentenceLength:
        tmp.append(newDataset2[i][2])
    else:
        count = 0
        listY2.append(tmp)
        tmp = []
        tmp.append(newDataset2[i][2])
    count += 1

In [17]:
X_train, X_test, y_train, y_test = train_test_split(np.array(listX), np.array(listY), test_size=0.1, random_state=420)
X_train2, X_test2, y_train2, y_test2 = train_test_split(np.array(listX2), np.array(listY2), test_size=0.1, random_state=42)

In [18]:
def checkNotIncludeds(TrainList, TestList):
    tmp = []
    tmp2 = []
    result = []
    for liste in TrainList:
        for element in liste:
            if element != 'O':
                tmp.append(element)
    for liste in TestList:
        for element in liste:
            if element != 'O':
                tmp2.append(element)
    tmp = (list(set(tmp)))
    tmp2 = (list(set(tmp2)))
    for element in tmp2:
        if (element in tmp) == False:
            result.append(element)
    return result

In [19]:
print(checkNotIncludeds(y_train, y_test))

['I-treffen,widerfahren,zustoßen', 'B-Pfeffer', 'I-Zutat-Schinken', 'B-Zubereitung,mischen,vermengen', 'B-Zutat-Spargel', 'B-verzehren', 'B-komplett,vollständig,vollzählig', 'I-Zutat-Spargel', 'B-gehören', 'B-vorschlagen', 'B-temperaturspezifisch', 'B-schauen', 'I-entfernen', 'B-Statistik', 'B-treffen,widerfahren,zustoßen', 'I-gehören', 'B-Zutat,Muskatnuss']


In [20]:
print(checkNotIncludeds(y_train2, y_test2))

['weglassen', 'roh', 'Schritte', 'Tomaten', 'ungekocht', 'Milch', 'weißer-Spargel']
